# TensorFlow 1.0 API使用

In [1]:
# 导入
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

1.13.1
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.3
numpy 1.16.4
pandas 0.24.2
sklearn 0.22.1
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


## 数据读取--fashion_mnist

In [2]:
# 导入数据集fashion mnist
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [6]:
# 数据归一化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1, 28 * 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1,1)).reshape(-1, 28 * 28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1,1)).reshape(-1, 28 * 28)

In [7]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

2.0231433 -0.8105136


## 计算图构建--实现全连接网络
使用基础的语法，实现图像分类的例子

先构建图，再传入数据，要用占位符来顶替数据的位置

In [8]:
hidden_units = [100, 100]
class_num = 10

# 占位符，顶替数据位置
x = tf.placeholder(tf.float32, [None, 28 * 28])
y = tf.placeholder(tf.int64, [None])

# 定义输入层
input_for_next_layer = x
# 定义隐藏层
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(
        input_for_next_layer, hidden_unit, activation=tf.nn.relu)
    
# 输出层
# last_hidden_output * W (logits) -> softmax -> prob概率
# 1. logit -> softmax -> prob
# 2. labels -> one_hot
# 3. calculate cross entropy
logits = tf.layers.dense(input_for_next_layer, class_num)
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)

# 得到预测值
prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(prediction, y)
# get accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# 运行一次train_op，网络就训练一次
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [9]:
print(x)
print(logits)

Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)


## 模型训练
session 是客户端（python）和C++运行时的连接，打开会话，就可以运行图了

In [10]:
# 初始化
init = tf.global_variables_initializer()
batch_size = 20
epochs = 10
train_steps_per_epoch = x_train_scaled.shape[0] // batch_size
valid_steps = x_valid_scaled.shape[0] // batch_size

# 验证
def eval_with_sess(sess, x, y, accuracy, images, labels, batch_size):
    eval_steps = images.shape[0] // batch_size
    eval_accuracies = []
    for step in range(eval_steps):
        batch_data = images[step * batch_size : (step + 1) * batch_size]
        batch_label = labels[step * batch_size : (step + 1) * batch_size]
        accuracy_val = sess.run(accuracy, feed_dict = {
            x: batch_data,
            y: batch_label
        })
        eval_accuracies.append(accuracy_val)
    return np.mean(eval_accuracies)

# 创建会话
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            # 取数据
            batch_data = x_train_scaled[step * batch_size : (step + 1) * batch_size]
            batch_label = y_train[step * batch_size : (step + 1) * batch_size]
            # 训练
            loss_val, accuracy_val, _ = sess.run(
                [loss, accuracy, train_op],
                feed_dict = {
                    x: batch_data,
                    y: batch_label
                })
            print('\r[Train] epoch:%d, step:%d, loss:%3.5f, accuracy:%2.2f' % (
                epoch, step, loss_val, accuracy_val), end='')
        # 验证
        valid_accuracy = eval_with_sess(sess, x, y, accuracy,
                                        x_valid_scaled, y_valid, batch_size)
        print('\t[Valid] acc:%2.2f' % (valid_accuracy))

[Train] epoch:0, step:2749, loss:0.38786, accuracy:0.80	[Valid] acc:0.86
[Train] epoch:1, step:2749, loss:0.23071, accuracy:0.85	[Valid] acc:0.87
[Train] epoch:2, step:2749, loss:0.19158, accuracy:0.95	[Valid] acc:0.88
[Train] epoch:3, step:2749, loss:0.15562, accuracy:0.90	[Valid] acc:0.88
[Train] epoch:4, step:2749, loss:0.20032, accuracy:0.90	[Valid] acc:0.88
[Train] epoch:5, step:2749, loss:0.12497, accuracy:0.90	[Valid] acc:0.88
[Train] epoch:6, step:2749, loss:0.10159, accuracy:0.95	[Valid] acc:0.88
[Train] epoch:7, step:2749, loss:0.07915, accuracy:0.95	[Valid] acc:0.88
[Train] epoch:8, step:2749, loss:0.04988, accuracy:1.00	[Valid] acc:0.88
[Train] epoch:9, step:2749, loss:0.10262, accuracy:0.90	[Valid] acc:0.88


## Dataset构建，再建模训练

### Dataset构建

In [16]:
y_train = np.asarray(y_train, dtype=np.int64)
y_valid = np.asarray(y_valid, dtype=np.int64)
y_test = np.asarray(y_test, dtype=np.int64)

In [17]:
def make_dataset(images, labels, epochs, batch_size, shuffle = True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [18]:
# test tf1.0 中 dataset是不可迭代的，会报错
batch_size = 20
epochs = 10
dataset = make_dataset(x_train_scaled, y_train, epochs=epochs, batch_size=batch_size)

# 报错，只能在eager mode下使用
# for data, label in dataset.take(1):
#     print(data)
#     print(label)

dataset.make_one_shot_iterator 的特点：
1. auto initialization
2. can't be re-initialized.
   make_initializable_iterator can be

In [19]:
# tf1.0 中正确使用
batch_size = 20
epochs = 10
dataset = make_dataset(x_train_scaled, y_train, epochs=epochs, batch_size=batch_size)

dataset_iter = dataset.make_one_shot_iterator()
x, y = dataset_iter.get_next()

with tf.Session() as sess:
    x_val, y_val = sess.run([x, y])
    print(x_val.shape)
    print(y_val.shape)

(20, 784)
(20,)


### 建模训练

In [20]:
hidden_units = [100, 100]
class_num = 10

# 定义输入层
input_for_next_layer = x
# 定义隐藏层
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(
        input_for_next_layer, hidden_unit, activation=tf.nn.relu)
    
# 输出层
logits = tf.layers.dense(input_for_next_layer, class_num)
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)

# 得到预测值
prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(prediction, y)
# get accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# 运行一次train_op，网络就训练一次
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [21]:
print(x)
print(logits)

Tensor("IteratorGetNext_1:0", shape=(?, 784), dtype=float32)
Tensor("dense_8/BiasAdd:0", shape=(?, 10), dtype=float32)


In [22]:
init = tf.global_variables_initializer()
train_steps_per_epoch = x_train_scaled.shape[0] // batch_size

# 创建会话
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            # 训练
            loss_val, accuracy_val, _ = sess.run(
                [loss, accuracy, train_op])
            print('\r[Train] epoch:%d, step:%d, loss:%3.5f, accuracy:%2.2f' % (
                epoch, step, loss_val, accuracy_val), end='')

[Train] epoch:9, step:2749, loss:0.33168, accuracy:0.90

## 另一种Dataset构建方式（推荐）
### Dataset构建

In [24]:
batch_size = 20
epochs = 10

# 构建dataset
images_placeholder = tf.placeholder(tf.float32, [None, 28 * 28])
labels_placeholder = tf.placeholder(tf.int64, [None, ])
dataset = make_dataset(images_placeholder, labels_placeholder,
                       epochs=epochs, batch_size=batch_size)

# 定义迭代器
dataset_iter = dataset.make_initializable_iterator()
x, y = dataset_iter.get_next()

with tf.Session() as sess:
    # 初始化
    sess.run(dataset_iter.initializer, feed_dict = {
        images_placeholder: x_train_scaled,
        labels_placeholder: y_train
    })
    x_val, y_val = sess.run([x, y])
    print(x_val.shape)
    print(y_val.shape)
    sess.run(dataset_iter.initializer, feed_dict = {
        images_placeholder: x_valid_scaled,
        labels_placeholder: y_valid
    })
    x_val, y_val = sess.run([x, y])
    print(x_val.shape)
    print(y_val.shape)

(20, 784)
(20,)
(20, 784)
(20,)
